In [ ]:

import requests
from PIL import Image
from io import BytesIO
import boto3
import os, dotenv

dotenv.load_dotenv()

In [ ]:
# Initialize Connection Stringss
aws_access_key_id = os.getenv('AWS_SECRET_PUBLIC_KEY')
aws_secret_access_key = os.getenv('AWS_SECRET_PRIVATE_KEY')
aws_bucket_name = os.getenv('AWS_BUCKET_NAME')
connection_string = f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_IP')}/{os.getenv('POSTGRES_DB')}"
print(aws_access_key_id, aws_secret_access_key, aws_bucket_name)
print(connection_string)

## Initiate AWS-S3 Session

In [ ]:
# Initiate AWS & S3 Session
aws_session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_SECRET_PUBLIC_KEY'),
    aws_secret_access_key=os.getenv('AWS_SECRET_PRIVATE_KEY')
)
aws_s3_session = aws_session.resource('s3')
aws_session_bucket = aws_s3_session.Bucket(aws_bucket_name)

In [ ]:
aws_s3_session.Bucket(aws_bucket_name).put_object(Key='images/test.txt', Body=b'Hello World!')

## Initialize Database

In [ ]:
import asyncpg
database = await asyncpg.create_pool(dsn=connection_string)

In [ ]:
result = await database.fetch('SELECT * FROM spottings')
image_urls = [row['imageUrl'] for row in result]
image_urls

## Download & Optimize Images

In [ ]:
import os, tempfile
# os.mkdir('images')

In [ ]:
print(len(image_urls))
for image in image_urls:
    image_loc = image
    print(image_urls.index(image) + 1, image)

    image_data = aws_s3_session.Object(aws_bucket_name, image).get()['Body'].read()
    image_io = BytesIO(image_data)
    image = Image.open(image_io)

    # Optimize Image
    # create temp file to append to IO
    with tempfile.NamedTemporaryFile(delete=False) as temp:
        image.save(temp.name, format="WebP", quality=10, optimize=True)        
        temp.close()

        with open(temp.name, 'rb') as optimized:
            optimized_image_io = BytesIO(optimized.read())
    os.remove(temp.name)
    
    # Grab Data from IO
    data = optimized_image_io.getvalue()
    
    # Save to Storage
    # with open(image_loc, 'wb') as f:
    #     f.write(data)
        
    # Uploads to S3
    aws_s3_session.Bucket(aws_bucket_name).put_object(Key=f'{image}', Body=data)
    # print(f"Optimized {image} to S3 on {aws_bucket_name}")
    
    # break